In [ ]:
import json
import os
from pathlib import Path

import json_tricks
import numpy as np

from alibi_detect.utils.saving import load_detector
from alibi_detect.od import OutlierVAE
from sklearn.preprocessing import StandardScaler

from ad_nids.dataset import Dataset
from ad_nids.ml import build_vae
from ad_nids.utils.misc import jsonify

In [ ]:
log_path = Path('/home/emikmis/data/nids/bad_logs/2019-12-20_09-55-58-952355_conf_f7f2b_002/')

In [ ]:
list(log_path.iterdir())

In [ ]:
with open(log_path/'config.json', 'r') as f:
    config = json.load(f)

In [ ]:
config

In [ ]:
dataset_path = Path('/home/emikmis/data/nids/processed/CTU_13_TRAIN_2-9_TEST_3_5T_BASIC/')
dataset = Dataset.from_path(dataset_path)

normal_batch = dataset.create_outlier_batch(train=True, perc_outlier=0)
X_train, y_train = normal_batch.data.astype(np.float32), normal_batch.target
scaler = StandardScaler().fit(X_train)

test_batch = dataset.create_outlier_batch(train=False, scaler=scaler)
X_test, y_test = test_batch.data.astype(np.float32)[:100], test_batch.target[:100]

In [ ]:
input_dim = X_train.shape[1]

vae = build_vae(config['hidden_dim'], config['latent_dim'],
                    config['num_hidden'], input_dim)
od_new = OutlierVAE(threshold=0.0, vae=vae, score_type='mse',
                latent_dim=config['latent_dim'], samples=config['samples'])

In [ ]:
od = load_detector(str(log_path/'detector'))

In [ ]:
preds = od.predict(X_test)